In [1]:
# Python import
import os
import copy
import random
import collections
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import svm
import warnings
import joblib
from sklearn.model_selection import train_test_split,RandomizedSearchCV
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


In [2]:
#Evaluate performance of model
def evaluate_performance(y_test, y_pred, y_prob):
    # AUROC
    auroc = metrics.roc_auc_score(y_test,y_prob)
    auroc_curve = metrics.roc_curve(y_test, y_prob)
    # AUPRC
    auprc=metrics.average_precision_score(y_test, y_prob) 
    auprc_curve=metrics.precision_recall_curve(y_test, y_prob)
    #Accuracy
    accuracy=metrics.accuracy_score(y_test,y_pred) 
    #MCC
    mcc=metrics.matthews_corrcoef(y_test,y_pred)
    
    recall=metrics.recall_score(y_test, y_pred)
    precision=metrics.precision_score(y_test, y_pred)
    f1=metrics.f1_score(y_test, y_pred)
    class_report=metrics.classification_report(y_test, y_pred,target_names = ["control","case"])

    model_perf = {"auroc":auroc,"auroc_curve":auroc_curve,
                  "auprc":auprc,"auprc_curve":auprc_curve,
                  "accuracy":accuracy, "mcc": mcc,
                  "recall":recall,"precision":precision,"f1":f1,
                  "class_report":class_report}
        
    return model_perf

In [3]:
# Output result of evaluation
def eval_output(model_perf,path):
    with open(os.path.join(path,"Evaluate_Result_TestSet.txt"),'w') as f:
        f.write("AUROC=%s\tAUPRC=%s\tAccuracy=%s\tMCC=%s\tRecall=%s\tPrecision=%s\tf1_score=%s\n" %
               (model_perf["auroc"],model_perf["auprc"],model_perf["accuracy"],model_perf["mcc"],model_perf["recall"],model_perf["precision"],model_perf["f1"]))
        f.write("\n######NOTE#######\n")
        f.write("#According to help_documentation of sklearn.metrics.classification_report:in binary classification, recall of the positive class is also known as sensitivity; recall of the negative class is specificity#\n\n")
        f.write(model_perf["class_report"])

In [4]:
# Plot AUROC of model
def plot_AUROC(model_perf,path):
    #get AUROC,FPR,TPR and threshold
    roc_auc = model_perf["auroc"]
    fpr,tpr,threshold = model_perf["auroc_curve"]
    #return AUROC info
    temp_df = pd.DataFrame({"FPR":fpr,"TPR":tpr})
    temp_df.to_csv(os.path.join(path,"AUROC_info.txt"),header = True,index = False, sep = '\t')
    #plot
    plt.figure()
    lw = 2
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='AUROC (area = %0.2f)' % roc_auc) 
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("AUROC of Models")
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(path,"AUROC_TestSet.pdf"),format = "pdf")

In [5]:
# Random seed
SEED = 100
random.seed(SEED)
np.random.seed(SEED)

warnings.filterwarnings(action='ignore')

# Output dir
output_dir = "./ML_Model_Output"
if not (os.path.exists(output_dir)):
    os.mkdir(output_dir)

In [6]:
# 合并所有样本
folder_path = '/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/NomalSamples/labels/'

# 获取该文件夹下所有 CSV 文件的文件名，并按文件名顺序排序
csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])

# 创建一个空的列表，用于存储所有的 DataFrame
dfs = []

# 依次读取每个 CSV 文件并将其添加到列表中
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)  # 读取 CSV 文件
    dfs.append(df)  # 将 DataFrame 添加到列表中

# 将所有的 DataFrame 按列合并
labels = pd.concat(dfs, axis=0)

# 设置文件夹路径
folder_path = '/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/NomalSamples/samples/'
# 获取该文件夹下所有 CSV 文件的文件名，并按文件名顺序排序
csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])
# 创建一个空的列表，用于存储所有的 DataFrame
dfs = []
# 依次读取每个 CSV 文件并将其添加到列表中
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)  # 读取 CSV 文件
    dfs.append(df)  # 将 DataFrame 添加到列表中
# 将所有的 DataFrame 按列合并
dataset = pd.concat(dfs, axis=0)


In [7]:
# 加载两个数据集，对样本数较多的数据集进行下采样平衡
#dataset = pd.read_csv('/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/SRR11004118_sample_prepared.csv')    
#labels = pd.read_csv('/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/SRR11004118_labels.csv')
dataset = pd.concat([dataset, labels['label']], axis=1)
df_y0 = dataset[dataset['label'] == 0]
df_y1 = dataset[dataset['label'] == 1]

# 确定两个子集中数量较少的那个
min_count = min(len(df_y0), len(df_y1))

# 从两个子集中随机选择等量的样本
df_y0_balanced = df_y0.sample(n=min_count, random_state=42) if len(df_y0) > min_count else df_y0
df_y1_balanced = df_y1.sample(n=min_count, random_state=42) if len(df_y1) > min_count else df_y1
# 合并这两个平衡后的子集
balanced_df = pd.concat([df_y0_balanced, df_y1_balanced])
# 打乱合并后的数据集的顺序
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
output_dir = './ML_Model_Output'

X = balanced_df.iloc[:, 1:-1].values
y = balanced_df.iloc[:, -1].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
%%time
#Construct Logistic Regression model
print("\n*** Logistic Regression  ***")

# Logistic Regression params
lr_param_dict = {
    "penalty":["l2"],
    "C":[1e-3, 5e-3, 1e-2, 0.05, 0.1, 0.5,1,5,10,50,100,500,1000],
    "solver":["liblinear"],
    "random_state":[SEED]
}

#Initiate model
lr_model = LogisticRegression()
#Adjust hyper-parameters with 5-fold cross validation
lr_rscv = RandomizedSearchCV(lr_model, lr_param_dict, n_iter=100,cv = 5,verbose = 0,
                          scoring = "roc_auc",random_state=SEED,n_jobs = 30)
lr_rscv.fit(x_train, y_train)  

#Evaluate best Lasso model
#Output path
path = os.path.join(output_dir,"LogisticRegression")
if not (os.path.exists(path)):
    os.mkdir(path)
    
# Model performance(AUROC) on cross-validation dataset
lr_cv_perf = np.array([ lr_rscv.cv_results_["split%s_test_score"%str(i)] for i in range(5)])[:,lr_rscv.best_index_]

#Get best model with score [max(mean(auc(5 cross validation)))]
lr_best_model = lr_rscv.best_estimator_
#Get predict_class(y_pred) and predict_probality_for_case(y_prob) of TestSet
y_pred = lr_best_model.predict(x_test)
y_prob = lr_best_model.predict_proba(x_test)[:,1]

#Get model performance
model_perf = evaluate_performance(y_test,y_pred,y_prob)
#Output result of evaluation
eval_output(model_perf,path)
#You can make bar plot consisted of accuracy,sensitivity,specificity,auroc,f1 score,MCC,precision,recall,auprc according to the "Evaluate_Result_TestSet.txt"
# Plot AUROC
plot_AUROC(model_perf,path)

#save model
joblib.dump(lr_best_model,os.path.join(path,"best_LogisticRegression_model.pkl"))

In [ ]:
%%time
#Construct SVM model
print("\n*** SVM ***")

# SVM params
SVM_param_dict = {
    'kernel':('linear', 'rbf'), 
    'C':[0.01,0.1,1,10, 100], 
    'gamma':[0.001, 0.005, 0.1 ,0.5,1, 2],
    "probability":[True],
    "random_state":[SEED]
}

#Initiate model
SVM_model = svm.SVC()
#Adjust hyper-parameters with 5-fold cross validation
SVM_rscv = RandomizedSearchCV(SVM_model, SVM_param_dict, n_iter=100,cv = 5,verbose = 0,
                          scoring = "roc_auc",random_state=SEED,n_jobs =30)
SVM_rscv.fit(x_train, y_train) 


#Evaluate best SVM model
#Output path
path = os.path.join(output_dir,"SVM")
if not (os.path.exists(path)):
    os.mkdir(path)
    
# Model performance(AUROC) on cross-validation dataset
SVM_cv_perf = np.array([ SVM_rscv.cv_results_["split%s_test_score"%str(i)] for i in range(5)])[:,SVM_rscv.best_index_]

#Get best model with score [max(mean(auc(5 cross validation)))]
svm_best_model = SVM_rscv.best_estimator_
#Get predict_class(y_pred) and predict_probality_for_case(y_prob) of TestSet
y_pred = svm_best_model.predict(x_test)
y_prob = svm_best_model.predict_proba(x_test)[:,1]

#Get model performance
model_perf = evaluate_performance(y_test,y_pred,y_prob)
#Output result of evaluation
eval_output(model_perf,path)
#You can make bar plot consisted of accuracy,sensitivity,specificity,auroc,f1 score,MCC,precision,recall,auprc according to the "Evaluate_Result_TestSet.txt"
# Plot AUROC
plot_AUROC(model_perf,path)

#save model
joblib.dump(svm_best_model,os.path.join(path,"best_SVM_model.pkl"))
#load model
#svm_best_model = joblib.load(os.path.join(path,"best_SVM_model.pkl"))
######################################

In [ ]:
%%time
# Construct Random Forest model
print("\n*** Random Forest  ***")

# Random Forest params
rf_param_dict = {
    "n_estimators": [10, 50, 100, 200, 500, 1000],
    "max_depth": [None, 10, 20, 50, 100],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['auto', 'sqrt', 'log2'],
    "random_state": [SEED]
}

# Initiate model
rf_model = RandomForestClassifier()
# Adjust hyper-parameters with 5-fold cross-validation
rf_rscv = RandomizedSearchCV(rf_model, rf_param_dict, n_iter=100, cv=5, verbose=0,
                             scoring="roc_auc", random_state=SEED, n_jobs=30)
rf_rscv.fit(x_train, y_train)

# Evaluate best Random Forest model
# Output path
path = os.path.join(output_dir, "RandomForest")
if not (os.path.exists(path)):
    os.mkdir(path)

# Model performance(AUROC) on cross-validation dataset
rf_cv_perf = np.array([rf_rscv.cv_results_["split%s_test_score" % str(i)] for i in range(5)])[:, rf_rscv.best_index_]

# Get best model with score [max(mean(auc(5 cross validation)))]
rf_best_model = rf_rscv.best_estimator_
# Get predict_class(y_pred) and predict_probability_for_case(y_prob) of TestSet
y_pred = rf_best_model.predict(x_test)
y_prob = rf_best_model.predict_proba(x_test)[:, 1]

# Get model performance
model_perf = evaluate_performance(y_test, y_pred, y_prob)
# Output result of evaluation
eval_output(model_perf, path)
# You can make bar plot consisted of accuracy, sensitivity, specificity, auroc, f1 score, MCC, precision, recall, auprc according to the "Evaluate_Result_TestSet.txt"
# Plot AUROC
plot_AUROC(model_perf, path)

# Save model
joblib.dump(rf_best_model, os.path.join(path, "best_RandomForest_model.pkl"))


In [ ]:
%%time
# Construct Random Forest model
print("\n*** Random Forest  ***")

# Random Forest params
rf_param_dict = {
    "n_estimators": [10, 50, 100, 200, 500, 1000],
    "max_depth": [None, 10, 20, 50, 100],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['auto', 'sqrt', 'log2'],
    "random_state": [SEED]
}

# Initiate model
rf_model = RandomForestClassifier()
# Adjust hyper-parameters with 5-fold cross-validation
rf_rscv = RandomizedSearchCV(rf_model, rf_param_dict, n_iter=1000, cv=5, verbose=0,
                             scoring="roc_auc", random_state=SEED, n_jobs=30)
rf_rscv.fit(x_train, y_train)

# Evaluate best Random Forest model
# Output path
path = os.path.join(output_dir, "RandomForest")
if not (os.path.exists(path)):
    os.mkdir(path)

# Model performance(AUROC) on cross-validation dataset
rf_cv_perf = np.array([rf_rscv.cv_results_["split%s_test_score" % str(i)] for i in range(5)])[:, rf_rscv.best_index_]

# Get best model with score [max(mean(auc(5 cross validation)))]
rf_best_model = rf_rscv.best_estimator_
# Get predict_class(y_pred) and predict_probability_for_case(y_prob) of TestSet
y_pred = rf_best_model.predict(x_test)
y_prob = rf_best_model.predict_proba(x_test)[:, 1]

# Get model performance
model_perf = evaluate_performance(y_test, y_pred, y_prob)
# Output result of evaluation
eval_output(model_perf, path)
# You can make bar plot consisted of accuracy, sensitivity, specificity, auroc, f1 score, MCC, precision, recall, auprc according to the "Evaluate_Result_TestSet.txt"
# Plot AUROC
plot_AUROC(model_perf, path)

# Save model
joblib.dump(rf_best_model, os.path.join(path, "best_RandomForest_model.pkl"))


In [ ]:
%%time
print("\n*** LightGBM ***")

# LightGBM params
lgb_param_dict = {
    "learning_rate":[0.1, 0.05, 0.02, 0.01],
    "num_leaves": range(10,36,5),
    "max_depth" : [2,3,4,5,10,20,40,50],
    "min_child_samples": range(1, 45, 2),
    "colsample_bytree" : [i / 10 for i in range(2,11)],
    "metric" : ["binary_logloss"],
    "n_jobs":[1],
    "n_estimators" : range(100,2500,100),
    "subsample" :  [i / 10 for i in range(2, 11)],
    "subsample_freq" : [0, 1, 2],
    "reg_alpha" : [0, 0.001, 0.005, 0.01, 0.1],
    "reg_lambda" : [0, 0.001, 0.005, 0.01, 0.1],
    "objective":["binary"],
    "random_state":[SEED]
}

#Initiate model
lgb_model = lgb.LGBMClassifier()
#Adjust hyper-parameters with 5-fold cross validation
lgb_rscv = RandomizedSearchCV(lgb_model, lgb_param_dict, n_iter=1000,cv = 5,verbose = 0,
                          scoring = "roc_auc",random_state=SEED,n_jobs = 30)
lgb_rscv.fit(x_train, y_train)   


#Evaluate best LightGBM model
#Output path
path = os.path.join(output_dir,"LightGBM")
if not (os.path.exists(path)):
    os.mkdir(path)
    
# Model performance(AUROC) on cross-validation dataset
lgb_cv_perf = np.array([ lgb_rscv.cv_results_["split%s_test_score"%str(i)] for i in range(5)])[:,lgb_rscv.best_index_]

#Get best model with score [max(mean(auc(5 cross validation)))]
lgb_best_model = lgb_rscv.best_estimator_
#Get predict_class(y_pred) and predict_probality_for_case(y_prob) of TestSet
y_pred = lgb_best_model.predict(x_test)
y_prob = lgb_best_model.predict_proba(x_test)[:,1]

#Get model performance
model_perf = evaluate_performance(y_test,y_pred,y_prob)
#Output result of evaluation
eval_output(model_perf,path)
#You can make bar plot consisted of accuracy,sensitivity,specificity,auroc,f1 score,MCC,precision,recall,auprc according to the "Evaluate_Result_TestSet.txt"
# Plot AUROC
plot_AUROC(model_perf,path)

#save model
joblib.dump(lgb_best_model,os.path.join(path,"best_LightGBM_model.pkl"))
#load modelq
#lgb_best_model = joblib.load(os.path.join(path,"best_LightGBM_model.pkl"))